In [12]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from pathlib import Path
import numpy as np
import seaborn as sns

In [13]:
folder = "data/comp1_2020_1/"
Path(folder).mkdir(parents=True, exist_ok=True)

In [14]:
with open(f'{folder}/dropout_df_total_and_per_week_dict_3_6.pkl', 'rb') as pklfile:
    df_total_and_per_week = pickle.load(pklfile)

In [15]:
exclude = df_total_and_per_week[(df_total_and_per_week['attempts_12'] == 0) & \
                                (df_total_and_per_week['attempts_13'] == 0) & \
                                (df_total_and_per_week['attempts_14'] == 0) & \
                                (df_total_and_per_week['attempts_15'] == 0)]
dtw2 = df_total_and_per_week.drop(index=exclude.index)
dtw2

,attempts_12,success_rate_12,avg_timedelta_to_solution_12,avg_timedelta_from_beginning_12,on_time_exercises_12,frequency_12,success_count_12,attempts_13,success_rate_13,avg_timedelta_to_solution_13,...,on_time_exercises_19,frequency_19,success_count_19,attempts,success_rate,avg_timedelta_to_solution,avg_timedelta_from_beginning,on_time_exercises,frequency,success_count
910,8,0.375000,65.386857,328.720191,3,1,3,24,0.291667,291.476823,...,6,1,6,32,0.312500,223.649833,519.849833,10,2,10
911,4,0.750000,132.823499,684.156833,3,1,3,36,0.194444,23916.274449,...,6,2,6,40,0.250000,16781.239164,17381.439164,10,3,10
912,6,0.500000,24.270101,207.270101,3,1,3,7,1.000000,0.000000,...,6,1,6,13,0.769231,7.281030,141.581030,10,2,10
914,9,0.333333,430.621847,918.621847,3,1,3,91,0.076923,40044.526822,...,6,2,6,100,0.100000,28160.355329,28983.855329,10,4,10
915,8,0.375000,124.739469,243.072802,3,1,3,12,0.416667,18.759911,...,5,1,5,20,0.400000,58.502245,167.252245,8,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,0,NaN,NaN,NaN,0,3,0,0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,0,3,0
1403,0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,...,5,1,5,0,NaN,NaN,NaN,0,0,0
1405,6,0.500000,32.428669,655.095336,0,2,3,22,0.181818,26859.988079,...,0,0,0,28,0.250000,15362.462618,15919.319761,0,7,7
1407,56,0.053571,2124.487458,3170.154125,0,1,3,0,NaN,NaN,...,0,0,0,56,0.053571,NaN,NaN,0,3,3


In [16]:
dtw2['success_count_label'] = dtw2['success_count_17'] + dtw2['success_count_19']

In [17]:
# Features and fill missing data
features_week7 = dtw2[['on_time_exercises_12',
                       'on_time_exercises_13',
                       'on_time_exercises_14',
                       'on_time_exercises_15',
                      ]].reset_index(drop=True)

# Target
outcome_week7 = dtw2['success_count_label'].reset_index(drop=True)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(features_week7, outcome_week7, test_size=0.2)
idx = np.array(X_train.index.tolist())

In [19]:
X_train_const = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_const).fit()
X_test_const = sm.add_constant(X_test)
y_pred_ols = model.predict(X_test_const)
rmse = mean_squared_error(y_test, y_pred_ols, squared=False)
print("RMSE: %.2f" % rmse)
model.summary()

RMSE: 3.67


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     success_count_label   R-squared:                       0.328
Model:                             OLS   Adj. R-squared:                  0.320
Method:                  Least Squares   F-statistic:                     39.40
Date:                 Mon, 29 Nov 2021   Prob (F-statistic):           7.25e-27
Time:                         18:41:10   Log-Likelihood:                -915.41
No. Observations:                  328   AIC:                             1841.
Df Residuals:                      323   BIC:                             1860.
Df Model:                            4                                         
Covariance Type:             nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    4.5547      0.409     11.140      0.000       3.750       5.359
on_time_exercises_12     0.6803      0.183      3.712      0.000       0.320       1.041
on_time_exercises_13     0.0581      0.110      0.528      0.598      -0.158       0.274
on_time_exercises_14     0.2178      0.098      2.214      0.028       0.024       0.411
on_time_exercises_15     0.5671      0.122      4.664      0.000       0.328       0.806
==============================================================================
Omnibus:                        7.872   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.020   Jarque-Bera (JB):                8.071
Skew:                          -0.365   Prob(JB):                       0.0177
Kurtosis:                       2.761   Cond. No.                         14.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
with open("%s/model_week7.pkl" % folder, "wb") as pklfile:
    pickle.dump(model, pklfile)

In [21]:
n_medio = 0
medio = 0
n_baixo = 0
baixo = 0
n_alto = 0
alto = 0
for n in range(len(y_test)):
    if y_pred_ols.tolist()[n] < 6.5:
        baixo += 1
        if y_test.tolist()[n] < 4:
            n_baixo += 1
    elif y_pred_ols.tolist()[n] < 9.5:
        medio += 1
        if y_test.tolist()[n] < 4:
            n_medio += 1
    else:
        alto += 1
        if y_test.tolist()[n] < 4:
            n_alto += 1
    
if baixo:
    print(f'Risco baixo: {n_baixo}/{baixo} ({100*n_baixo/baixo}%)')
if medio:
    print(f'Risco médio: {n_medio}/{medio} ({100*n_medio/medio}%)')
if alto:
    print(f'Risco alto: {n_alto}/{alto} ({100*n_alto/alto}%)')

Risco baixo: 11/18 (61.111111111111114%)
Risco médio: 5/18 (27.77777777777778%)
Risco alto: 2/46 (4.3478260869565215%)


In [22]:
y_pred_ols.unique()

array([11.87121846, 11.92926959,  5.12181918,  6.82303553, 11.5953275 ,
       10.11195098,  5.6439462 ,  8.98577093,  4.6127982 ,  7.90678783,
        6.59560018,  6.48816795,  9.04637889,  4.84500274,  9.88841648,
       11.24898522,  4.72890047, 10.62223063, 11.30507182, 11.71142976,
        9.25462319,  4.55474707,  8.18812562,  9.60293   , 10.056084  ,
        9.48205854, 10.40439081,  7.12732272, 11.63901392, 11.14435765,
       11.52291165,  6.48598379, 10.22805325, 10.98549444,  8.34791431,
       11.65337863,  4.83063803,  6.65365132])